# Connect to ES

In [1]:
import elasticsearch
from elasticsearch import Elasticsearch
from decouple import config

In [2]:
elasticsearch.__version__

(8, 3, 3)

In [3]:
client = Elasticsearch(
    "https://ml-elasticsearch.kcp.bbp.epfl.ch:443",
    basic_auth=("elastic", config('ES_PASS')),
    verify_certs=False,
)

/home/reissant/miniforge3/envs/py10/lib/python3.10/site-packages/elasticsearch/_sync/client/__init__.py:395: SecurityWarning: Connecting to 'https://ml-elasticsearch.kcp.bbp.epfl.ch:443' using TLS with verify_certs=False is insecure
  _transport = transport_class(


In [8]:
client.info()

/home/reissant/miniforge3/envs/py10/lib/python3.10/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ml-elasticsearch.kcp.bbp.epfl.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


ObjectApiResponse({'name': 'elasticsearch-coordinating-0', 'cluster_name': 'elastic', 'cluster_uuid': 'dv5oiXnCRJiB8nhV7a81RA', 'version': {'number': '8.3.3', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': '801fed82df74dbe537f89b71b098ccaff88d2c56', 'build_date': '2022-07-23T19:30:09.227964828Z', 'build_snapshot': False, 'lucene_version': '9.2.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

# Create indices

## mapping articles

In [10]:
client.indices.create(
    index="articles",
    body={
        "settings": {"number_of_shards": 2,
                     "number_of_replicas": 1},
        "mappings": {
            "dynamic": "strict",
            "properties": {
                "article_id": {"type": "keyword"},
                "doi": {"type": "keyword"},
                "pmc_id": {"type": "keyword"},
                "pubmed_id": {"type": "keyword"},
                "title": {"type": "text"},
                "authors": {"type": "text"},
                "abstract": {"type": "text"},
                "journal": {"type": "keyword"},
                "publish_time": {"type": "date", "format": "yyyy-MM-dd"},
                "license": {"type": "keyword"},
                "is_english": {"type": "boolean"},
            }
        },
    }
)

/tmp/ipykernel_1937749/4048237628.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  client.indices.create(
/home/reissant/miniforge3/envs/py10/lib/python3.10/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ml-elasticsearch.kcp.bbp.epfl.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'articles'})

## mapping paragraphs

In [ ]:
client.indices.create(
    index="paragraphs",
    body={
        "settings": {"number_of_shards": 2,
                     "number_of_replicas": 1},
        "mappings": {
            "dynamic": "strict",
            "properties": {
                "article_id": {"type": "keyword"},
                "section_name": {"type": "keyword"},
                "paragraph_id": {"type": "short"},
                "text": {"type": "text"},
                "is_bad": {"type": "boolean"},
                "emb": {
                    "type": "dense_vector",
                    "dims": 384,
                    "index": True,
                    "similarity": "dot_product"
                }
            }
        },
    }
)

## check indices

In [9]:
indices = client.indices.get_alias().keys()
sorted(indices)

/home/reissant/miniforge3/envs/py10/lib/python3.10/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ml-elasticsearch.kcp.bbp.epfl.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


['arxiv_dense']